**INITIAL DESCRIPTION**

Here are some helpers defined for you:

`solve` runs a `clingo` sub process, places the results in a file called dump.lp (that you can use for debugging), and the processes it into a Python dictionary

`parse_json_result` processes the result as a json file and does the processing into the dictionary file

`pretty_print` takes the resultant dictionary and produces a text grid visualization using emoji

In [1]:
import json
import collections
import subprocess
import random
import sys


def solve(args):
    """Run clingo with the provided argument list and return the parsed JSON result."""

    print_args = ['clingo'] + list(args) + [' | tr [:space:] \\\\n | sort ']
    args = ['clingo', '--outf=2'] + args + ["--sign-def=rnd","--seed="+str(random.randint(0,1<<30))]
    with subprocess.Popen(
        ' '.join(args),
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        shell=True
    ) as clingo:
        outb, err = clingo.communicate()
    if err:
        print(err)
    out = outb.decode("utf-8")
    with open('dump.lp', 'w') as outfile:
        result = json.loads(out)
        witness = result['Call'][0]['Witnesses'][-1]['Value']
        for atom in sorted(witness):
            outfile.write(atom + '\n')
    
    return parse_json_result(out)


def parse_json_result(out):
    """Parse the provided JSON text and extract a dict
    representing the predicates described in the first solver result."""

    result = json.loads(out)

    assert len(result['Call']) > 0
    assert len(result['Call'][0]['Witnesses']) > 0

    witness = result['Call'][0]['Witnesses'][0]['Value']

    class identitydefaultdict(collections.defaultdict):
        def __missing__(self, key):
            return key

    preds = collections.defaultdict(set)
    env = identitydefaultdict()

    for atom in witness:
        if '(' in atom:
            left = atom.index('(')
            functor = atom[:left]
            arg_string = atom[left:]
            try:
                preds[functor].add( eval(arg_string, env) )
            except TypeError:
                pass # at least we tried...

        else:
            preds[atom] = True
    return dict(preds)


def pretty_print(world):
    import functools
    width = functools.reduce(lambda x,y: max(x,y),world['width'])
    height = functools.reduce(lambda x,y: max(x,y),world['height'])
    grid = [[ ' ' for w in range(width)] for h in range(height)]

    legend = {'grass':'🌿',
    'flower':'🌼',
    'bridge':'🌉',
    'water':'🌊',
    'tree':'🌲',
    'door':'🚪',
    'wall':'🗄️',
    'bed':'🛏️',
    'floor':'🔲'}


    for t in world['terrain']:
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
    for t in world.get('construction',[]):
        xx = int(t[0])
        yy = int(t[1])
        type = t[2]
        grid[yy-1][xx-1] = legend[type]
        
    return ('\n'.join([''.join(r) for r in grid]))

Now we can run it and print the results!

Note: the use of `-c` lets us change a `#const` defined in the ASProlog code, so that we can dynamically resize the resultant map without needing to modify our code.

In [2]:
%%time

width = 12
height = 12
world = solve(['world_gen.lp','-c','max_width={}'.format(width),'-c','max_height={}'.format(height)])

print(pretty_print(world))


🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿
🌿🌿🌊🌊🌿🌿🌊🌊🌿🌊🌊🌿
🌿🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊🌊
🌿🌊🌊🌊🌊🌿🌉🌉🌉🌿🌊🌊
🌲🌿🌊🌊🌿🌿🌊🌊🌼🌿🌿🌊
🌼🌿🌿🌿🌿🌊🌊🌊🌊🌉🌊🌊
🌿🌿🌿🌿🌿🌊🌊🌊🌊🌉🌊🌊
🌿🌿🌿🌿🌿🌊🌊🌊🌊🌉🌊🌊
🌿🌿🌿🌿🌿🌊🌊🌊🌿🌿🌊🌊
🌿🌿🌿🌿🌿🌊🌊🌊🌿🌊🌊🌊
🌿🌿🌿🌿🌿🌿🌊🌊🌊🌊🌊🌿
🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿🌿
CPU times: user 17.1 ms, sys: 7.29 ms, total: 24.4 ms
Wall time: 1.77 s


Now its your turn!

You will add code to generate a house in your own file house_gen.lp .  If you run your code at the same time as world_gen.lp, it will generate the terrain at the same time as your house is generated (guaranteeing that the terrain that is generated is capable of supporting your house).

Your code will create a `construction` predicate of the form:

`construction(X_Position, Y_Position, Construction_Type)`

where `Construction_Type` is one of `door, wall, floor, bed`.

This is a free-form assignment, but you have a few constraints that you should satisfy:

* Your house should contain at least 1 door, 1 bed, all non-bed tiles should be floor, and the perimeter of the house should be walls
* It should not place 2 doors side by side
* Doors shouldn't be placed in corners, but should be able to face out to any cardinal direction
* Doors should open to grass or flowers, not water, bridges, or trees
* All pieces of your house should be on top of grass or flowers, not water, bridges, or trees
* The position of your house shouldn't be hardcoded
* A wall piece must touch (in cardinal directions) only 2 doors or walls

* Bonus credit: 10 points -- Your houses can be of variable width and height
* Bonus credit: 10 points -- Your houses can be in shapes that aren't just rectangles

In [ ]:
width = 12
height = 12
world = solve(['world_gen.lp','house_gen.lp','-c','max_width={}'.format(width),'-c','max_height={}'.format(height)])

print(pretty_print(world))